In [ ]:
# Visual formatting options

# Sets the display to 98% of the window's max width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

<div class="alert alert-block alert-info">

## Table of Contents <a name="ToC"></a>

- [1. TO-DO and Functions list](#todo) <br>
   - [1.1 Importing libraries & Reading in data](#importing) <br>
- [2. Read and/or generate databases](#allframes) <br>
- [3. General functions](#general) <br>
- [4. Job Seeker functions](#seeker) <br>
- [5. Job Recruiter functions](#recruiter) <br>



<div class="alert alert-block alert-danger">

# TO-DO LIST <a name="todo"></a>
[Return to Table of Contents](#ToC) <br>

- Build class wrappers
  - User class
  - Job class (Listing Creation, Listing Read, Job Application Creation, Job Application Read) 
      - Search subclass?

- Run obfuscation for password
  -  Cryptographic library instead?
- Pull file system manipulation for saving Jupyter notebooks from previous programs
- Blow shit up
- Use something like getpass (library) to hide input responses

## Functions list
***
### General functions
- Create account
- Login to account
- Update account
  - ___Should applicants be able to change their status from Seeker to Recruiter / vice versa?___

***
### Recruiter functions
- Create Job Categories
- Create Job Listing
- Manage Job Categories
- Manage Job Listing
- Publish Job
  - ___Should this have a secondary password checker? Should anything have it?___
- Check Job Applicants
- Headhunter Interview
- Job Applicant Interview

***
### Seeker functions
- Manage Skillset
- Search for Job
  - Apply for Job
- Respond to Interview Invitation

***
### System functions
- Score matching for job search / headhunter search

<div class="alert alert-block alert-info">

## Imports and global declarations <a name="importing"></a>
[Return to Table of Contents](#ToC) <br>


In [1]:
# -------
# Imports
# -------

# Top-level, and always useful
import re
import os
from collections import Counter
from IPython.display import display
import sys
from copy import copy

# Misc
import random

# Dataframes, yo!
import pandas as pd
import numpy as np

# ---------
# Functions
# ---------
# Flatten list of lists
def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

# Append items to list in-line, returning a list with appended items
def ret_append(lst, item):
    lst.append(item)
    return lst

# -----------------
# Misc declarations
# -----------------
# Pandas declarations

# Hide Chained Assignment (SettingWithCopy warning)
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Global variables
current_UID = ''
password_attempts = 0
all_frames = [accts]

In [ ]:
# Demo account user ID
accts.UID[accts.email == 'tester@jobapps.org'][0]

<div class="alert alert-block alert-info">

## Read and/or import databases <a name="allframes"></a>
[Return to Table of Contents](#ToC) <br>


In [7]:
# Database file names
accounts_db = "test_db.csv"

### Account details database

In [2]:
# Checks whether database is present in CWD; if not, generate a new database

if assert accounts_db in os.listdir(os.getcwd()):
    accts = pd.read_csv(accounts_db)
else:
    # Test dataframe - replace with CSV file on network drive where possible
    demo_seeker = {'firstName':'Ted',
            'lastName':'Testable',
            'password':['testerted'],
           'isSeeker':True,
           'UID':'tetest0001s',
           'email':'tester@jobapps.org',
           'skills':[['Systems Testing', 'Network Security', 'Data Science']]
           }

    demo_recruiter = {'firstName':'Rick',
            'lastName':'Recruitable',
            'password':['recruiterrick'],
           'isSeeker':False,
           'UID':'rirecr0001r',
           'email':'recruiter@jobapps.org',
           'skills':[[]]
           }
    accts = pd.concat([pd.DataFrame(demo_seeker), pd.DataFrame(demo_recruiter)])
    accts = accts[['UID', 'firstName', 'lastName', 'email', 'password', 'isSeeker', 'skills']]
    accts.to_csv("test_db.csv", index=False)

### Job categories dataframe

In [ ]:
#demo = {'catID':'00001', 'catName':'Data Science'}
#job_cats = pd.DataFrame(demo)



### Job Listings dataframe

### Interview dataframe

In [5]:
accts.to_csv("test_db.csv", index=False)

<div class="alert alert-block alert-info">

## General functions <a name="general"></a>
[Return to Table of Contents](#ToC) <br>

In [ ]:
# Create a new account.
# Takes full name, email, new password (inc. checks), whether the user is a seeker or a recruiter, and generates a unique
# user ID.

def genAcct(database):
    # ------------------------------------------
    # Request & store user details
    # ------------------------------------------
    
    # Names
    first = input("What is your first name? ")
    last = input("What is your surname? ")

    # Email and password; use first and last name + email to check if the applicant is already in the system
    email = input("What is your email address?")
    if len(accts[(accts.email == email)]) != 0:
        print("\nWARNING: An account with the email is recorded in the system - please login, or contact us on [GENERIC_EMAIL] if you"+
              " have forgotten your password.")
        exit
    check_state = False
    while check_state == False:
        password = input("Please type your password: ")
        pass_check = input("Please type your password again to confirm: ")
        if password == pass_check:
            check_state = True
        else:
            print("Passwords did not match; please try again.")
    
    # ------------------------------------------
    # Request job seeker / recruiter status
    # ------------------------------------------
    
    def Seeker():
        isSeeker = input("Are you a jobseeker? If so, please type True. If not, please type False. ")
        if isSeeker.lower() == 'true':
            isSeeker = True
        elif isSeeker.lower() == 'false':
            isSeeker = False
        else: # Recursive request if not True/False
            print("Not a recognised input; please try again.")
            isSeeker = Seeker()
        return isSeeker
    isSeeker = Seeker()
    
    # ------------------------------------------
    # Generate unique user ID - append 'x's if chars are missing from name, and then 000x numbering.
    # ------------------------------------------
    
    # Name substring
    id_string = first.lower()[0:2] + last.lower()[0:4]
    if len(id_string) < 6:
        id_string += (6-len(id_string))*'x'
    
    # Counts the occurence count of the user ID, add one, and use this for numbering.
    id_string += (str(len(accts[accts['UID'].str.contains(id_string)])+1).zfill(4))
    
    # 's'/'r' seeker/recruiter substring
    if isSeeker:
        id_string += ('s')
    else:
        id_string += ('r')
    
    # ------------------------------------------
    # Generates new account details to database after checking data
    # ------------------------------------------
    
    # Check data
    assert isinstance(first, str)
    assert isinstance(last, str)
    assert isinstance(isSeeker, bool)
    
    # Generate new row for database
    details = {'firstName':first,
    'lastName':last,
    'password':password,
   'isSeeker':isSeeker,
   'UID':id_string,
   'email':email,
   'skills':[],
   }
    database.loc[len(database)] = details
    
    # Update CSV file storing database
    accts.to_csv(accounts_db, index=False)
    
    return "New account added."

In [ ]:
def login(attempts = 0):
    # Carry over however many password attempts have been made previously.
    password_attempts = attempts
    login_email = input("Please provide your email address.")
    
    # If email not present, either retry login or go to account generation. Return to login if unclear.
    if len(accts[accts.email == login_email]) == 0:
        print("Email account not detected.")
        option = input("Either type RETRY to attempt again, or NEW ACCOUNT to proceed to account creation.")
        if option.lower() == 'retry':
            return login()
        elif option.lower() == 'new account':
            return genAcct(accts)
        else:
            print("Input not recognised. Returning you to the login page.")
            return login()
    
    # If account detected, request & check password for email. If successful, set global var current_UID = UID from email.
    if len(accts[accts.email == login_email]) == 1:
        if checkEmailPassword(login_email, accts):
            global current_UID
            current_UID = accts.UID[accts.email == login_email][0]
            password_attempts = 0
        else: # If password incorrect, offer another try. Login attempts are limited to 5 times.
            password_attempts += 1
            while password_attempts <= 5:
                print("Password incorrect. Please try again. You have", 5-password_attempts, "attempts left.")
                return login(password_attempts)
            else:
                return("Password incorrect. You have tried to login with an incorrect password too many times. Please "+
                     "contact support at [GENERIC_EMAIL].")
    
    # If email not detected, offer a retry, an exit, or new account creation.
    else:
        while new_acct.lower not in ['yes', 'exit', 'retry']:
            new_acct = input("Email not detected. Please type YES to proceed to account creation, RETRY to attempt login"+
                             "again, otherwise type EXIT to close the platform.")
            if new_acct.lower() == 'yes':
                genAcct(accts)
            elif new_acct.lower() == 'exit':
                logout()
            elif new_acct.lower() == 'retry':
                login()

# Uses UID from the accounts database to set UID value, which can be carried across all databases if necessary.
def checkEmailPassword(email_checked, database):
    uid = accts.UID[accts.email == email_checked][0]
    state = False
    login_password = input("Please provide your password.")
    if login_password == database.password[database.UID == uid][0]:
        state = True
    else:
        state = False
    return state

# Reset global var current_UID to empty string. Returns login function
def logout():
    global current_UID
    current_UID = ''
    print("Logged out. Please login to continue.")
    return login()                


In [8]:
# Update user email, password, or Job Seeker status.
def updateAccount():
    if current_UID == '':
        login()
    details_dict = {'1':'email', '2':'password', '3':'isSeeker', '4':'EXIT'}
    print("Please select from the following options:\n"+
         "1. Change email\n"+
         "2. Change password\n"+
         "3. Change role (Job Seeker or Recruiter)\n"+
         "4. Exit")
    choice = ''
    # Take input prompt
    while choice not in ['1', '2', '3', '4']:
        choice = input("Please select the number option that you wish to proceed with.")
    
    # Use dict key:val to edit dataframe.
    # If just updating email or password, changes only need to be made to the accts dataframe.
    if choice in ['1', '2']:
        print("Change", details_dict[1], "selected.")
        details = input("What would you like to change this to?")
        accts[details_dict[choice]][accts.UID == current_UID] = details
        return updateAccount()
    
    # If updating isSeeker status, all occurences of the user ID will have to change in all dataframes.
    #elif choice == '3':
    #    old_uid = copy(current_UID)
    #    print("Changing Job Seeker/Recruiter status selected. Please enter 's' for Job Seeker, or 'r' for Job Recruiter (without quotation marks).")
    #    details = input("What would you like to change your status to?")
    #    if details in ['r', 's']:
    #        new_uid = old_uid[:-1] + details
    #        for dataframe in all_frames: # Iterates through all dataframes, changing any occurence of the current UID to an editted version where the trailing 's'/'r' is updated.
    #            dataframe.UID[dataframe.UID = old_uid] = new_uid
    else:
        pass
    # Update CSV file storing database
    accts.to_csv(accounts_db, index=False)
    return

<div class="alert alert-block alert-info">

## Job Seeker functions <a name="seeker"></a>
[Return to Table of Contents](#ToC) <br>

In [ ]:
# Prompt users to provide their skill sets. Use Levenshtein checker to compare to existing skills (inc. lemmatization?)
def updateSkills():
    # Pull system-wide skills and compare using Levenshtein distance.
    global_skills = []
    for skill_list in accts.skills.tolist():
        for skill in skill_list:
            global_skills.append(skill)
    
    # Pull the skills the user wants to add to their skillset, and then check if close terms exist
    skill_store = input("Please provide your skills, separating each skillset by a comma.").split(',')
    for skill in skill_store:
        print(skill)
        ## LEVENSTEIN CHECK AGAINST CURRENT_SKILLS FOR EACH SKILL;
        ## REPACE OPTION IF EDIT DISTANCE IS MINOR TO EXISTING SKILL
        
    # Generate new frame, then update the details in the matching index with the new frame.
    #user_skills = flatten(accts.skills[accts.UID == current_UID].tolist())
    new_data = pd.DataFrame({'skills':skill_store}, index=[accts[accts.UID == current_UID].index[0]])
    accts.update(new_data)
    
    # Update CSV file storing database
    accts.to_csv(accounts_db, index=False)
    return "Skills updated."

In [ ]:
def jobSearch():
    return("Job search functionality not implemented yet.")

In [ ]:
def checkIntInvitations():
    return("Interview invitation request checking not implemented yet.")

In [ ]:
# Assert userid flag is for 
def menu():
    if current_UID[-1] == 's':
        seekerMenu()
    elif current_UID[-1] == 'r':
        recruiterMenu()
    elif current_UID[-1] == 'a':
        adminMenu()
    else:
        print("Account UID is non-standard - please contact support at [GENERIC_EMAIL] as soon as possible."+
              "You will now be logged out")
        logout()

In [ ]:
def seekerMenu():
    print("Available options:\n"+
          "1. Update Skills\n"+
          "2. Search for Jobs\n"+
         "3. See Current Interview Invitations\n"+
         "4. Exit")
    request = input("Please select the number option that you wish to proceed with.")
    if request == '1':
        updateSkills()
    elif request == '2':
        jobSearch()
    elif request == '3':
        checkIntInvitations()
    elif request == '4':
        logout()
        return
    else:
        print("Input not recognised.")
        return seekerMenu()
            
def recruiterMenu():
    return("Not enabled yet")
    
def adminMenu():
    return("Not enabled yet")

<div class="alert alert-block alert-info">

## Job Recruiter functions <a name="recruiter"></a>
[Return to Table of Contents](#ToC) <br>

In [ ]:
# Generate new job categories. Need to import Levenshtein checker to compare to existing jobs
def genJobCategory():
    pass

<div class="alert alert-block alert-info">

## Testing section <a name="testing"></a>
[Return to Table of Contents](#ToC) <br>


Please restrict testing to this section of the notebook.

In [ ]:
display(accts)
genAcct(accts)

In [ ]:
login()

In [ ]:
current_UID = 'tetest0001s'

In [ ]:
updateAccount()

In [ ]:
menu()